# Testing out NLI 

Notebook to get the baseline usage of the model `tals/albert-xlarge-vitaminc` from [HuggingFace](https://huggingface.co/tals/albert-xlarge-vitaminc).

Here are a few text examples for sanity checks

In [ ]:
evidence_claim_pairs = [
    {
        "evidence": "The new policy has led to a significant decrease in crime rates.",
        "claim": "The new policy reduces crime.",
        "label": "Supports"
    },
    {
        "evidence": "There are no studies showing a direct link between the policy and crime rates.",
        "claim": "The new policy has a high impact on crime rates.",
        "label": "Not enough info"
    },
    {
        "evidence": "Crime rates have increased since the policy was implemented.",
        "claim": "The new policy reduces crime.",
        "label": "Refutes"
    }
]


## Imports and Model Loading

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F


# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("tals/albert-xlarge-vitaminc")
model = AutoModelForSequenceClassification.from_pretrained("tals/albert-xlarge-vitaminc")


## Prediction pipeline

Takes in two strings, evidence and claim. Outputs the tuple of the label and corresponding prediction score

In [ ]:
def predict_label(text_a, text_b):
    """Predicts the relationship between a claim and evidence. 

    Args:
        text_a (str): The evidence statement.
        text_b (str): The claim statement.

    Returns:
        str: The predicted relationship label ("SUPPORTS", "REFUTES", or "NOT ENOUGH INFO").
        float: The confidence score of the prediction.
    """

    # Tokenize the input
    inputs = tokenizer(
        text_a, text_b,
        return_tensors='pt',  # Return PyTorch tensors
        padding=True,         # Pad to the longest sequence in the batch
        truncation=True       # Truncate to the model's max length
    )

    # Make predictions
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Convert logits to probabilities
    probabilities = F.softmax(logits, dim=1)
    
    # Get the predicted class and its score
    predicted_class = torch.argmax(probabilities, dim=1).item()
    predicted_score = probabilities[0, predicted_class].item()

    # Label mapping
    label_map = {0: "SUPPORTS", 1: "REFUTES", 2: "NOT ENOUGH INFO"}  # Updated label mapping

    return label_map[predicted_class], predicted_score  # Return label and score


Silly example:

In [ ]:
# Example usage
label, score = predict_label("The sky is blue.", "The color of the pineapple is orange.")
print(f"Predicted class: {label}, Score: {score}")



Another one

In [ ]:
label, score = predict_label(
    "The sky is light blue.", # evidence
    "The color of the sky is blue." # claim
    )
print(f"Predicted class: {label}, Score: {score}")


## Testing
Lets try for our original examples

In [ ]:
for pair in evidence_claim_pairs:
    print(
        predict_label(pair['evidence'],pair['claim']))
    print("ground truth:", pair['label'])

That looks good! Yay! 

Lets move onto working this for value kaleidescope.